In [4]:
import pandas as pd
import xgboost as xgb
from xgboost import XGBClassifier
import datetime; import pytz
import matplotlib as plt
from scipy.special import softmax
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split # (*arrays, **options)
import numpy as np
from sklearn.metrics import log_loss
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

from joblib import dump, load
import joblib
import os
from sklearn.metrics import confusion_matrix, mean_squared_error
from sklearn.model_selection import KFold, train_test_split, GridSearchCV

In [5]:
datadir = '/opt/data'
localdir = '/opt/program'
tripsdf = pd.read_csv(f'{datadir}/2013-07 - Citi Bike trip data.csv')
stationsdf = pd.read_csv(f'{localdir}/datas/stations/stations-2018-12-04-c.csv',
                        index_col=0)

In [6]:
os.getcwd()

'/opt/program'

In [7]:
# load model from 2020-06-10 notebook
bundle = joblib.load('/opt/program/artifacts/2020-06-11T041641Z/bundle.joblib')

In [8]:
bundle

{'notebook': '2020-06-10-again',
 'model': XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
               colsample_bynode=1, colsample_bytree=1, gamma=0,
               learning_rate=0.1, max_delta_step=0, max_depth=3,
               min_child_weight=1, missing=nan, n_estimators=100, n_jobs=1,
               nthread=None, objective='multi:softprob', random_state=0,
               reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
               silent=None, subsample=1, verbosity=1),
 'actuals': array([60, 60, 60, ...,  2,  2,  2]),
 'predictions': array([46, 46, 46, ...,  2,  2,  2]),
 'confusion_matrix': array([[ 193,   64,   14, ...,    0,    0,  150],
        [  27, 1167,   16, ...,    0,    0,   28],
        [   4,    6,  379, ...,    0,    0,  236],
        ...,
        [   0,    0,   17, ...,    0,    0,   46],
        [  67,  599,   24, ...,    0,    0,   78],
        [  19,    9,  196, ...,    0,    0, 2017]]),
 'walltime_train': '19min 45s',
 'pre

In [20]:
# quick manual accuracy computation
bundle['actuals'].shape, bundle['predictions'].shape
correct = len([i for i, _ in enumerate(bundle['actuals'])
              if bundle['actuals'][i] == bundle['predictions'][i]
              ])
print({'overall accuracy': correct/len(bundle['actuals']), 
       'num_correct': correct, 'total': len(bundle['actuals'])})
print(bundle['confusion_matrix'].shape)

confusion_diagonal = [bundle['confusion_matrix'][i][j]
                                  for i in range(54)
                                  for j in range(54)
                                  if i == j]
print(confusion_diagonal)
print('confusion, sum, diagonal', sum(confusion_diagonal))

{'overall accuracy': 0.1541161182619253, 'num_correct': 64992, 'total': 421708}
(54, 54)
[193, 1167, 379, 0, 0, 1406, 0, 1039, 30505, 0, 685, 557, 67, 62, 49, 0, 4534, 0, 2583, 109, 0, 0, 2766, 2003, 0, 0, 0, 236, 0, 2722, 0, 0, 612, 6064, 581, 0, 0, 0, 0, 24, 0, 0, 0, 0, 0, 0, 4632, 0, 0, 0, 0, 0, 0, 2017]
confusion, sum, diagonal 64992


In [25]:
model = bundle['model']
print(len(model.feature_importances_))
print(sorted(model.feature_importances_))

78
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.00057195826, 0.00096085493, 0.0019978876, 0.0028608162, 0.003014325, 0.0031509278, 0.005542396, 0.0055443724, 0.0056226645, 0.0064963517, 0.0066661313, 0.007307959, 0.007578683, 0.0077737863, 0.008284873, 0.00866166, 0.009315344, 0.010423137, 0.010517005, 0.010585468, 0.010727769, 0.012014797, 0.012143584, 0.012632706, 0.01298739, 0.013684955, 0.014033192, 0.014308239, 0.014311185, 0.016608043, 0.017402876, 0.017734889, 0.017809916, 0.017948298, 0.018543119, 0.01898507, 0.019225849, 0.02069202, 0.020770036, 0.021067692, 0.02152036, 0.022673236, 0.024081945, 0.02418218, 0.02421692, 0.026871085, 0.027934289, 0.02844202, 0.030187974, 0.032351416, 0.033994444, 0.035514593, 0.03653184, 0.036576174, 0.044999287, 0.04772858, 0.05568539]


In [23]:
# 78 features
model.classes_

array([ 0,  1,  2,  4,  5,  7,  8, 11, 12, 13, 14, 15, 17, 18, 19, 21, 23,
       24, 25, 26, 27, 30, 32, 33, 34, 36, 37, 39, 40, 42, 43, 44, 45, 46,
       47, 48, 49, 50, 51, 52, 53, 57, 58, 60, 62, 63, 64, 65, 66, 67, 68,
       71, 72, 73])

NameError: name 'prepare_data' is not defined

In [29]:
import fresh.utils as fu
from importlib import reload
reload(fu)

<module 'fresh.utils' from '/opt/program/fresh/utils.py'>

In [31]:
X, y, neighborhoods = fu.prepare_data(tripsdf, stationsdf)
# = sorted(stationsdf.neighborhood.unique().tolist())



In [34]:
bundle['preproc']['one_hot_enc'].categories

[['Alphabet City',
  'Battery Park City',
  'Bedford-Stuyvesant',
  'Bloomingdale',
  'Boerum Hill',
  'Bowery',
  'Broadway Triangle',
  'Brooklyn Heights',
  'Brooklyn Navy Yard',
  'Carnegie Hill',
  'Carroll Gardens',
  'Central Park',
  'Chelsea',
  'Chinatown',
  'Civic Center',
  'Clinton Hill',
  'Cobble Hill',
  'Columbia Street Waterfront District',
  'Downtown Brooklyn',
  'Dumbo',
  'East Harlem',
  'East Village',
  'East Williamsburg',
  'Financial District',
  'Flatiron District',
  'Fort Greene',
  'Fulton Ferry District',
  'Garment District',
  'Governors Island',
  'Gowanus',
  'Gramercy Park',
  'Greenpoint',
  'Greenwich Village',
  "Hell's Kitchen",
  'Hudson Square',
  'Hunters Point',
  'Kips Bay',
  'Korea Town',
  'Lenox Hill',
  'Lincoln Square',
  'Little Italy',
  'Long Island City',
  'Lower East Side',
  'Lower Manhattan',
  'Meatpacking District',
  'Midtown',
  'Midtown East',
  'Midtown West',
  'Murray Hill',
  'NoHo',
  'NoMad',
  'Nolita',
  'Park S

In [35]:
len(bundle['preproc']['one_hot_enc'].categories[0]), len(bundle['preproc']['one_hot_enc'].categories[1])

(75, 3)

In [36]:
features = ([f'feature_start_neighborhood={x}' for x in bundle['preproc']['one_hot_enc'].categories[0]]
           + [f'feature_gender={x}' for x in bundle['preproc']['one_hot_enc'].categories[1]])
print(features)

['feature_start_neighborhood=Alphabet City', 'feature_start_neighborhood=Battery Park City', 'feature_start_neighborhood=Bedford-Stuyvesant', 'feature_start_neighborhood=Bloomingdale', 'feature_start_neighborhood=Boerum Hill', 'feature_start_neighborhood=Bowery', 'feature_start_neighborhood=Broadway Triangle', 'feature_start_neighborhood=Brooklyn Heights', 'feature_start_neighborhood=Brooklyn Navy Yard', 'feature_start_neighborhood=Carnegie Hill', 'feature_start_neighborhood=Carroll Gardens', 'feature_start_neighborhood=Central Park', 'feature_start_neighborhood=Chelsea', 'feature_start_neighborhood=Chinatown', 'feature_start_neighborhood=Civic Center', 'feature_start_neighborhood=Clinton Hill', 'feature_start_neighborhood=Cobble Hill', 'feature_start_neighborhood=Columbia Street Waterfront District', 'feature_start_neighborhood=Downtown Brooklyn', 'feature_start_neighborhood=Dumbo', 'feature_start_neighborhood=East Harlem', 'feature_start_neighborhood=East Village', 'feature_start_nei

In [39]:
importances = model.feature_importances_
labeled_importances = [[features[i], importances[i]] for i in range(78)]
sorted(labeled_importances, key=lambda x:x[1])

[['feature_start_neighborhood=Bloomingdale', 0.0],
 ['feature_start_neighborhood=Broadway Triangle', 0.0],
 ['feature_start_neighborhood=Carnegie Hill', 0.0],
 ['feature_start_neighborhood=Carroll Gardens', 0.0],
 ['feature_start_neighborhood=Cobble Hill', 0.0],
 ['feature_start_neighborhood=East Harlem', 0.0],
 ['feature_start_neighborhood=East Williamsburg', 0.0],
 ['feature_start_neighborhood=Governors Island', 0.0],
 ['feature_start_neighborhood=Gowanus', 0.0],
 ['feature_start_neighborhood=Greenpoint', 0.0],
 ['feature_start_neighborhood=Hunters Point', 0.0],
 ['feature_start_neighborhood=Lenox Hill', 0.0],
 ['feature_start_neighborhood=Long Island City', 0.0],
 ['feature_start_neighborhood=Prospect Heights', 0.0],
 ['feature_start_neighborhood=Prospect Park', 0.0],
 ['feature_start_neighborhood=Red Hook', 0.0],
 ['feature_start_neighborhood=Stuyvesant Heights', 0.0],
 ['feature_start_neighborhood=Sunset Park', 0.0],
 ['feature_start_neighborhood=Upper East Side', 0.0],
 ['feature

In [ ]:
# Perhaps indeed the starting locations which are so un-important, basically have just way too many
# destinations, or maybe there are just not enough trips involving those stations.
# But anyway, I think ultimately the importances which are there 
#    across different cross validation folds are the ones to focus on

In [40]:
# retry that multi-logloss ...
rng = np.random.RandomState(31337)
indices1 = []
kf = KFold(n_splits=2, shuffle=True, random_state=rng)
for train_index, test_index in kf.split(X):
    indices1.append([train_index, test_index])

indices2 = []
kf = KFold(n_splits=2, shuffle=True, random_state=rng)
for train_index, test_index in kf.split(X):
    indices2.append([train_index, test_index])
    


In [44]:
# hmm doesnt look the same though, guess that random seed doesnt work deterministically?
indices1[0][:5] , indices2[0][:5]


([array([     0,      2,      3, ..., 843411, 843412, 843415]),
  array([     1,      7,      8, ..., 843410, 843413, 843414])],
 [array([     5,      7,      9, ..., 843412, 843413, 843414]),
  array([     0,      1,      2, ..., 843408, 843411, 843415])])

In [51]:
#log_loss(y_true_enc, y_prob, labels=self.labels)
#help(log_loss)
bundle['preproc']['le'].classes_.shape

(75,)

In [54]:
X_transformed = bundle['preproc']['one_hot_enc'].transform(X[:1000])
y_prob = model.predict_proba(X_transformed)

In [57]:
y_prob[0].shape

(54,)

In [65]:
# ok got to clear up this inconsistency.. so, len(bundle['preproc']['one_hot_enc'].categories[0]) => 75
# but there are I believe only 54 neighborhoods actually represented in the data itself,

from collections import Counter
print(len(bundle['preproc']['one_hot_enc'].categories[0]))
print(len(dict(Counter([x[0] for x in X]))))

75
54


In [66]:
len(Counter(y))

54

In [ ]:
# yea so 54 in X and y... but i used stationsdf w/ 75 neighborhoods, 
# so at the very least I should not be using that if it ends up indeed , 
# blowing up the one hot encoder to 75 , ... this might be why we have those start neighborhoods, 
# with  0 importance, since there was just no data for them 